# Task 4: Conversational Interface - Chat with Llama 3 and Titan Premier LLMs

In this notebook, you build a chatbot using the llama3-8b-instruct and titan-text-premier Foundation Models (FMs) in Amazon Bedrock.

Conversational interfaces such as chatbots and virtual assistants can enhance the user experience for your customers. Chatbots use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries. You can use chatbots in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. Users can access them through various channels such as websites, social media platforms, and messaging apps.

- **Chatbot (Basic)**, Zero Shot chatbot with a FM model
- **Chatbot using prompt**, template(LangChain) - Chatbot with some context provided in the prompt template
- **Chatbot with persona**, Chatbot with defined roles. i.e. Career Coach and Human interactions
- **Contextual-aware chatbot**, Passing in context through an external file by generating embeddings.

## LangChain framework for building Chatbot with Amazon Bedrock

In conversational interfaces such as chatbots, remembering previous interactions becomes highly important, both at a short-term and long-term level.

The LangChain framework provides memory components in two forms. First, LangChain provides helper utilities for managing and manipulating previous chat messages. These are designed to be modular. Secondly, LangChain provides easy ways to incorporate these utilities into chains, allowing you to easily define and interact with different types of abstractions, which make powerful chatbots easy for you to build.

## Building a Chatbot with Context - Key Elements

The first process in building a context-aware chatbot is to generate embeddings for the context. Typically, you have an ingestion process which runs through your embedding model and generates the embeddings, which will then be stored in a vector store. In this notebook, you use the Titan Embeddings model for this. The second process is user request orchestration, interaction, invoking, and returning the results. This involves orchestrating the user request, interacting with the necessary models/components to gather information, invoking the chatbot to formulate a response, and then returning the chatbot's response back to the user.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio.



## Task 4.1: Environment setup

In this task, you set up your environment.

In [2]:
#ignore warnings and create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


In [3]:
# format instructions into a conversational prompt
from typing import Dict, List

def format_instructions(instructions: List[Dict[str, str]]) -> List[str]:
    """Format instructions where conversation roles must alternate system/user/assistant/user/assistant/..."""
    prompt: List[str] = []
    for instruction in instructions:
        if instruction["role"] == "system":
            prompt.extend(["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        elif instruction["role"] == "user":
            prompt.extend(["<|start_header_id|>user<|end_header_id|>\n", (instruction["content"]).strip(), " <|eot_id|>"])
        else:
            raise ValueError(f"Invalid role: {instruction['role']}. Role must be either 'user' or 'system'.")
    prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(prompt)

## Task 4.2: Using CoversationChain from LangChain to start the conversation

In this task, you enable the chatbot to carry conversational context across multiple interactions with users. Having a conversational memory is crucial for Chatbots to hold meaningful, coherent dialogues over time.

You implement conversational memory capabilities by building on top of LangChain's ConversationChain class. The ConversationChain acts as a persistent memory module, allowing to store and retrieve historical statements made by both the user and the chatbot agent.


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** The model outputs are non-deterministic.

In [4]:
#model configuration
from langchain.chains import ConversationChain
from langchain_aws import BedrockLLM
from langchain.memory import ConversationBufferMemory


titan_llm = BedrockLLM(model_id="amazon.titan-text-premier-v1:0", client=bedrock_client, model_kwargs={
    "maxTokenCount":512,
    "stopSequences":[],
    "temperature":0,
    "topP":0.9
    })

llama3_llm = BedrockLLM(model_id="meta.llama3-8b-instruct-v1:0", client=bedrock_client, model_kwargs={
        "max_gen_len": 512,
        "temperature": 0,
        "top_p": 1,
    })
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llama3_llm, verbose=True, memory=memory
)

instructions = [{"role": "user", "content": "Hi there!"}]

try:
    print(conversation.predict(input=format_instructions(instructions)))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: <|start_header_id|>user<|end_header_id|>
Hi there! <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI:

> Finished chain.
 Ah, hello! It's great to chat with you. I'm an AI designed to assist and communicate with humans. I've been trained on a vast amount of text data, which allows me to understand and respond to a wide range of questions and topics. I'm constantly learning and improving my abilities, so please bear with me if I make any mistakes. What would you like to talk about?


### New Questions

The model has responded with an initial message. Now, you ask it a few questions.

In [5]:
#new questions
instructions = [{"role": "user", "content": "Give me a few tips on how to start a new garden."}]
print(conversation.predict(input=format_instructions(instructions)))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: <|start_header_id|>user<|end_header_id|>
Hi there! <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI:  Ah, hello! It's great to chat with you. I'm an AI designed to assist and communicate with humans. I've been trained on a vast amount of text data, which allows me to understand and respond to a wide range of questions and topics. I'm constantly learning and improving my abilities, so please bear with me if I make any mistakes. What would you like to talk about?
Human: <|start_header_id|>user<|end_header_id|>
Give me a few tips on how to start a new garden. <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI:

> Finished chain.
  Starting a new

### Build on the questions

Now, ask a question without mentioning the word garden to see if the model can understand the previous conversation.

In [6]:
# build on the questions
instructions = [{"role": "user", "content": "bugs"}]
print(conversation.predict(input=format_instructions(instructions)))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: <|start_header_id|>user<|end_header_id|>
Hi there! <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI:  Ah, hello! It's great to chat with you. I'm an AI designed to assist and communicate with humans. I've been trained on a vast amount of text data, which allows me to understand and respond to a wide range of questions and topics. I'm constantly learning and improving my abilities, so please bear with me if I make any mistakes. What would you like to talk about?
Human: <|start_header_id|>user<|end_header_id|>
Give me a few tips on how to start a new garden. <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI:   Starting a new garden! Exciting! 

### Finishing this conversation

In [7]:
# finishing the conversation
instructions = [{"role": "user", "content": "That's all, thank you!"}]
print(conversation.predict(input=format_instructions(instructions)))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: <|start_header_id|>user<|end_header_id|>
Hi there! <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI:  Ah, hello! It's great to chat with you. I'm an AI designed to assist and communicate with humans. I've been trained on a vast amount of text data, which allows me to understand and respond to a wide range of questions and topics. I'm constantly learning and improving my abilities, so please bear with me if I make any mistakes. What would you like to talk about?
Human: <|start_header_id|>user<|end_header_id|>
Give me a few tips on how to start a new garden. <|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI:   Starting a new garden! Exciting! 

## Task 4.3: Chatbot using prompt template (LangChain)

In this task, you use the default PromptTemplate that is responsible for the construction of this input. LangChain provides several classes and functions to make constructing and working with prompts easy.

In [8]:
#default prompt template
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

chat_history = []

# turn verbose to true to see the full logs and documents
qa= ConversationChain(
    llm=llama3_llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

print(f"ChatBot:DEFAULT:PROMPT:TEMPLATE: is ={qa.prompt.template}")

ChatBot:DEFAULT:PROMPT:TEMPLATE: is =The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [9]:
#  prompt for a conversational agent
def format_prompt(actor:str, input:str):
    formatted_prompt: List[str] = []
    if actor == "system":
        prompt_template="""<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    elif actor == "user":
        prompt_template="""<|start_header_id|>{actor}<|end_header_id|>\n{input}<|eot_id|>"""
    else:
        raise ValueError(f"Invalid role: {actor}. Role must be either 'user' or 'system'.")   
    prompt = PromptTemplate.from_template(prompt_template)     
    formatted_prompt.extend(prompt.format(actor=actor,input=input))
    formatted_prompt.extend(["<|start_header_id|>assistant<|end_header_id|>\n"])
    return "".join(formatted_prompt)

In [10]:
# chat user experience
import ipywidgets as ipw
from IPython.display import display, clear_output

class ChatUX:
    """ A chat UX using IPWidgets
    """
    def __init__(self, qa, retrievalChain = False):
        self.qa = qa
        self.name = None
        self.b=None
        self.retrievalChain = retrievalChain
        self.out = ipw.Output()


    def start_chat(self):
        print("Starting chat bot")
        display(self.out)
        self.chat(None)


    def chat(self, _):
        if self.name is None:
            prompt = ""
        else: 
            prompt = self.name.value
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you , that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)
                try:
                    if self.retrievalChain:
                        result = self.qa.invoke({'question': prompt})
                    else:
                        result = self.qa.invoke({'input': format_prompt("user",prompt)}) #, 'history':chat_history})
                except:
                    result = "No answer"
                thinking.value=""
                print(f"AI:{result}")
                self.name.disabled = True
                self.b.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.b = ipw.Button(description="Send")
                self.b.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.b)))

Next, start a chat.

In [11]:
# start chat
chat = ChatUX(qa)
chat.start_chat()

Starting chat bot


Output()

## Task 4.4: Chatbot with persona

In this task, Artificial Intelligence(AI) assistant plays the role of a career coach. Role play dialogue requires pre-populating the conversation with a user message before starting the chat. A ConversationBufferMemory is used to pre-populate the dialog.

In [12]:
# conversation context as a career coach
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message(format_prompt("user","Context: You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know."))
conversation = ConversationChain(
     llm=llama3_llm, verbose=True, memory=memory
)

print(conversation.predict(input=format_prompt("user","What are the career options in AI?")))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: <|start_header_id|>user<|end_header_id|>
Context: You will be acting as a career coach. Your goal is to give career advice to users. For questions that are not career related, don't provide advice. Say, I don't know.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Human: <|start_header_id|>user<|end_header_id|>
What are the career options in AI?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

AI:

> Finished chain.
 Ah, what a great question! As a career coach, I'm excited to share with you the numerous career options in AI. With the rapid growth of AI and machine learning, the job market is evolving rapidly. Here are some of the most in-demand r

Now, ask a question that is not within this persona's specialty. The model should not answer that question and should give a reason for that.

In [13]:
# question with different context
conversation.verbose = False
print(conversation.predict(input=format_prompt("user","How to fix my car?")))

  I'm a career coach, not a mechanic! I don't have the expertise to provide advice on fixing cars. I can suggest searching online for tutorials, consulting a professional mechanic, or checking your car's manual for guidance. If you have a specific question about a career-related topic, I'd be happy to help. Otherwise, I'm not the right AI for the job.


## Task 4.5 Chatbot with Context

In this task, you ask the chatbot to answer questions based on context that was passed to it. You take a CSV file and use the Titan embeddings model to create a vector representing that context. This vector is stored in Facebook AI Similarity Search (FAISS). When the chatbot is asked a question, you will pass this vector back to the chatbot and have it retrieve the answer using the vector.

### Titan embeddings Model

Embeddings represent words, phrases, or any other discrete items as vectors in a continuous vector space. This allows machine learning models to perform mathematical operations on these representations and capture semantic relationships between them.

You use embeddings for the Retrieval-Augmented Generation (RAG) [document search capability](https://labelbox.com/blog/how-vector-similarity-search-works/).

In [14]:
# model configuration
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate

br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

### FAISS as VectorStore

In order to use embeddings for search, you need a store that can efficiently perform vector similarity searches. In this notebook, you use FAISS, which is an in-memory store. To permanently store vectors, you can use Knowledge Bases for Amazon Bedrock, pgVector, Pinecone, Weaviate, or Chroma.

The LangChain VectorStore APIs are available [here](https://python.langchain.com/v0.2/docs/integrations/vectorstores/).

In [15]:
# vector store
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

loader = CSVLoader("./rag_data/Amazon_SageMaker_FAQs.csv") # --- > 219 docs with 400 chars
documents_aws = loader.load() #
print(f"documents:loaded:size={len(documents_aws)}")

docs = CharacterTextSplitter(chunk_size=2000, chunk_overlap=400, separator=",").split_documents(documents_aws)

print(f"Documents:after split and chunking size={len(docs)}")
vectorstore_faiss_aws = None
try:
    
    vectorstore_faiss_aws = FAISS.from_documents(
        documents=docs,
        embedding = br_embeddings, 
        #**k_args
    )

    print(f"vectorstore_faiss_aws:created={vectorstore_faiss_aws}::")

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

documents:loaded:size=153
Documents:after split and chunking size=154
vectorstore_faiss_aws:created=<langchain_community.vectorstores.faiss.FAISS object at 0x7f61b84b5210>::


### Run a quick low code test 

You can use a Wrapper class provided by LangChain to query the vector database store and return the relevant documents. This runs a QA Chain with all default values.

In [16]:
# wrapper store faiss
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss_aws)
print(wrapper_store_faiss.query("R in SageMaker", llm=titan_llm))

R is supported with Amazon SageMaker. You can use R within SageMaker notebook instances, which include a preinstalled R kernel and the reticulate library. Reticulate offers an R interface for the Amazon SageMaker Python SDK, enabling ML practitioners to build, train, tune, and deploy R models.


### Chatbot application

For the chatbot, you need context management, history, vector stores, and many other components. You start by building a ConversationalRetrievalChain.

This uses conversation memory and RetrievalQAChain, which allow for passing in chat history that can be used for follow-up questions.

Set verbose to `True` to see everything going on behind the scenes.

In [17]:
# pass chat history for follow-up questions
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

memory_chain = ConversationBufferMemory(memory_key="chat_history", input_key="question", return_messages=True)
chat_history=[]

### Parameters used for ConversationRetrievalChain

- **Retriever:** You used `VectorStoreRetriever`, which is backed by a `VectorStore`. To retrieve text, there are two search types to choose from: `"similarity"` or `"mmr"`. `search_type="similarity"` uses similarity search in the retriever object, where it selects text chunk vectors that are most similar to the question vector.

- **Memory:** A memory chain stores the history.

- **Condense_question_prompt:** Given a user's question, you use the previous conversation and that question to construct a standalone question.

- **Chain_type:** If the chat history is long and exceeds the context limit, set this parameter to stuff, refine, map_reduce, or map-rerank.

If the question asked falls outside the scope of the context, the model will reply that it doesn't know the answer.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** To understand how the chain functions, uncomment the verbose=True line. This will print out detailed logs.

In [18]:
# turn verbose to true to see the full logs and documents
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chains import ConversationalRetrievalChain


qa = ConversationalRetrievalChain.from_llm(
    llm=titan_llm, 
    retriever=vectorstore_faiss_aws.as_retriever(), 
    memory=memory_chain,
    chain_type='stuff', 
    verbose=False
)

qa.combine_docs_chain.llm_chain.prompt = PromptTemplate.from_template("""
{context}:

Use at maximum 3 sentences to answer the question.If the context does not contain information to answer the
question, please state that "I could not find an exact answer to the question."

{question}:


Answer:
""")

Next, start a chat

In [19]:
chat = ChatUX(qa, retrievalChain=True)
chat.start_chat()

Starting chat bot


Output()


You have utilized Titan LLM to create a conversational interface with following patterns:

- Chatbot (Basic - without context)
- Chatbot using prompt template(Langchain)
- Chatbot with personas
- Chatbot with context

### Try it yourself

- Change the prompts to your specific usecase and evaluate the output of different models.
- Play with the token length to understand the latency and responsiveness of the service.
- Apply different prompt engineering principles to get better outputs.

### Cleanup

You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file and continue with **Task 5**.